In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [154]:
df = pd.read_csv("/Users/sa21/Desktop/Financial-Fraud-Project/data/Raw/PS_20174392719_1491204439457_log.csv")
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [155]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [156]:
df1= df.drop(['nameOrig', 'nameDest','oldbalanceDest', 'newbalanceDest', 'isFlaggedFraud'], axis=1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   oldbalanceOrg   float64
 4   newbalanceOrig  float64
 5   isFraud         int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 291.3+ MB


In [157]:
df1['transaction_difference'] = df1['oldbalanceOrg'] - df1['newbalanceOrig']
df1['mismatch_amount'] = df1['transaction_difference'] != df1['amount']

In [158]:
df1.isnull().sum()

step                      0
type                      0
amount                    0
oldbalanceOrg             0
newbalanceOrig            0
isFraud                   0
transaction_difference    0
mismatch_amount           0
dtype: int64

In [159]:
# Convert categorical 'type' to dummy variables
df1 = pd.get_dummies(df1, columns=['type'], drop_first=True)


In [160]:
bool_cols = df1.select_dtypes(include='bool').columns

df1[bool_cols] = df1[bool_cols].astype(int)


In [161]:
df1.columns

Index(['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'isFraud',
       'transaction_difference', 'mismatch_amount', 'type_CASH_OUT',
       'type_DEBIT', 'type_PAYMENT', 'type_TRANSFER'],
      dtype='object')

In [162]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   step                    int64  
 1   amount                  float64
 2   oldbalanceOrg           float64
 3   newbalanceOrig          float64
 4   isFraud                 int64  
 5   transaction_difference  float64
 6   mismatch_amount         int64  
 7   type_CASH_OUT           int64  
 8   type_DEBIT              int64  
 9   type_PAYMENT            int64  
 10  type_TRANSFER           int64  
dtypes: float64(4), int64(7)
memory usage: 534.0 MB


In [163]:
df1.to_csv('/Users/sa21/Desktop/Financial-Fraud-Project/data/Processed/cleaned_fraud_data.csv', index=False)


In [172]:
from sklearn.model_selection import train_test_split
X = df1.drop('isFraud', axis=1)
y = df1['isFraud']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
